### Projeto Final

### Bibliotecas

In [3]:
import numpy as np
import pandas as pd
import requests 

### Função Alerta

In [70]:
#!pip install plyer
from plyer import notification

def alerta(nivel, base, etapa):
    if nivel == 1:
        Nivel = 'Alerta Baixo'
        etapa = 'Extração'

    if nivel == 2:
        Nivel = 'Alerta Médio'
        etapa = 'Normalização'

    if nivel == 3:
        Nivel = 'Alerta Alto'
        etapa = 'Regras'
    
    notification.notify(title = f'ATENÇÃO: {Nivel}', 
                message = f'Falha: Carregamento da base  base de dados "{base}" \n Etapa: "{etapa}"', 
                app_name = 'Teste',
                timeout = 10)
    
alerta(3, 'Clientes', '3')

### Função para chamar API

In [88]:
def chamada_API(url):
    response = requests.get(url, verify=False)
    
    if response.status_code == 200:
        data = response.json()
        return pd.DataFrame(data)
    else:
        alerta(1, f"Acesso a API, erro = {response.status_code}",'')
        return response.status_code
        
    

### Bancos

In [94]:
bancos = chamada_API('https://brasilapi.com.br/api/banks/v1')
bancos.head(5)

c:\Users\A0045759\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'brasilapi.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,ispb,name,code,fullName
0,00000000,BCO DO BRASIL S.A.,1.0,Banco do Brasil S.A.
1,00000208,BRB - BCO DE BRASILIA S.A.,70.0,BRB - BANCO DE BRASILIA S.A.
2,00038121,Selic,NaN,Banco Central do Brasil - Selic
3,00038166,Bacen,NaN,Banco Central do Brasil
4,00122327,SANTINVEST S.A. - CFI,539.0,"SANTINVEST S.A. - CREDITO, FINANCIAMENTO E INV..."


### Taxas

In [97]:
taxas = chamada_API('https://brasilapi.com.br/api/taxas/v1')
taxas

c:\Users\A0045759\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'brasilapi.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,nome,valor
0,Selic,10.50
1,CDI,10.40
2,IPCA,3.93


### PIX

In [101]:
pix = chamada_API('https://brasilapi.com.br/api/pix/v1/participants')
pix.tail()

c:\Users\A0045759\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'brasilapi.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,ispb,nome,nome_reduzido,modalidade_participacao,tipo_participacao,inicio_operacao
822,95213211,"COOPERATIVA DE CRéDITO, POUPANçA E INVESTIMENT...",COOP SICREDI CAMINHO DAS ÁGUAS RS,PDCT,IDRT,2020-11-03T09:30:00.000Z
823,95424891,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENT...",COOP SICREDI VALE DO RIO PARDO,PDCT,IDRT,2020-11-03T09:30:00.000Z
824,95594941,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENT...",COOP SICREDI REG CENTRO RS/MG,PDCT,IDRT,2020-11-03T09:30:00.000Z
825,97259253,COOPERATIVA E ECONOMIA E CRÉDITO MÚTUO DOS MÉD...,CECM MÉDICOS PLANALTO MÉDIO RS - CREDISIS/CRED...,PDCT,IDRT,2021-10-04T12:53:10.567Z
826,97489280,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENT...",COOP SICREDI INTEGRAÇÃO BAHIA,PDCT,IDRT,2020-11-03T09:30:00.000Z
